In [1]:
from jira import JIRA, JIRAError
import getpass, re, time

print("Логин:", getpass.getuser())
j = JIRA(server="https://jira.wargaming.net", basic_auth=(getpass.getuser(), getpass.getpass('Пароль: ')))

Логин: s_litvinchuk
Пароль: ········


In [2]:
T = time.localtime() # struct_time
Ts = time.mktime(time.localtime())
#print("Now:", time.strftime("%Y-%m-%d %H:%M:%S", T) )

Tstart = time.strftime("%Y-%m-%dT08:00:00:00+0000",time.localtime(Ts - 60*60*24 *(T.tm_wday+7) ))
Tstop  = time.strftime("%Y-%m-%dT08:00:00:00+0000",time.localtime(Ts - 60*60*24 * T.tm_wday    ))
#Tstart = "2017-02-20T08:00:00.000+0000"
#Tstop  = "2017-02-27T08:00:00.000+0000"

print(Tstart[:10], '-', Tstop[:10], '\n' )

ISSUES = {"Game": "WSR-49208",
          "Bill": "WSR-49209",
          "Tech": "WSR-49210",
          "Rule": "WSR-49211" }
All_Time = 0

for dep in ISSUES:
    print(dep)
    staffer = dict()
    main_task = j.issue(ISSUES[dep])
    subtasks = [subt.key for subt in main_task.fields.subtasks[-2:]] # -2: = два последних сабтаска
    
    for issue_id in subtasks:
        issue = j.issue(issue_id)
        print('\t', issue_id, issue.fields.summary)
        
        for worklog in j.worklogs(issue_id): # issue.fields.worklog.worklogs returns only 20 fields.
            if Tstart <= worklog.started < Tstop:
                #print(worklog.created)
                try:
                    name = str(worklog.author.displayName)
                    staffer[name] = staffer.setdefault(name, 0) + worklog.timeSpentSeconds
                except:
                    print(worklog.author)
                    continue
    print( "{0:<26}  {1:<10} {2:>6}".format(" ", "Время(ч)", "ЧЧ:ММ" ) )
    for k in staffer:
        H = round(staffer[k]/3600, 2)
        hhmm = "{1}:{2:0>2}".format(k, staffer[k]//3600, round((staffer[k]%3600)/60) )
        print("{0:>26}  {1:^10} {2:>6}".format(k, H, hhmm) )
    
    Stuffer_Time = sum(staffer.values())
    H = Stuffer_Time
    hhmm = "{0}:{1:0>2}".format( H//3600, round((H%3600)/60 )) # округляем минуты, чтобы учесть влияние секунд
    print( "{0:>26}  {1:^10} {2:>6}\n".format("Итого:", round(H/3600, 2), hhmm ) )
    
    All_Time += Stuffer_Time
    
print("\nВсего времени по всем отделам: {0}ч \t [{1}:{2:0>2}]".format(round(All_Time/3600, 2), All_Time//3600, round((All_Time%3600)/60) ) )

2017-07-17 - 2017-07-24 

Bill
	 WSR-52530 [BS] Логирование работы в группе ЦПП ВК. Июль 2017
	 WSR-52884 [BS] Логирование работы в группе ЦПП ВК. Август 2017
                            Время(ч)    ЧЧ:ММ
         Dmitriy Z             0.43      0:26
           Vitali T            0.4       0:24
              Irina K          0.48      0:29
           Andrei B            1.3       1:18
        Viktoryia N            0.23      0:14
       Aleksandr R             0.68      0:41
           Maksim S            0.17      0:10
         Elena L               1.07      1:04
        Dzianis V              0.35      0:21
           Pavel Y             1.65      1:39
             Siarhei L         0.18      0:11
       Vitali M                0.72      0:43
            Darya C            1.17      1:10
         Svetlana B            0.53      0:32
        Aleksey T              1.77      1:46
  Anastasiya S                 0.42      0:25
        Dzmitry B              0.9       0:54
            I

In [3]:
# BACKUP: Упрощённый вариант.
Tstart = "2017-03-20T08:00:00.000+0000"
Tstop  = "2017-03-27T08:00:00.000+0000"

ISSUES = {"Game": ['WSR-49405','WSR-49876', 'WSR-50144', 'WSR-50145', 'WSR-50202' ],}

for dep in ISSUES:
    print(dep)
    staffer = dict()
    for issue_id in ISSUES[dep]:
        issue = j.issue(issue_id)
        print('\t', issue_id, issue.fields.summary)
        
        for worklog in j.worklogs(issue_id): # issue.fields.worklog.worklogs returns only 20 fields.
            if Tstart <= worklog.started < Tstop:
                #print(worklog.created)
                try:
                    name = str(worklog.author.displayName)
                    staffer[name] = staffer.setdefault(name, 0) + worklog.timeSpentSeconds
                except:
                    print(worklog.author)
                    continue
    print( "{0:<26}  {1:<10} {2:>6}".format(" ", "Время(ч)", "ЧЧ:ММ" ) )
    for k in staffer:
        #print(k, round(staffer[k]/3600, 2), 'h')
        H = round(staffer[k]/3600, 2)
        hhmm = "{1}:{2:0>2}".format(k, staffer[k]//3600, round((staffer[k]%3600)/60) )
        print("{0:>26}  {1:^10} {2:>6}".format(k, H, hhmm) )
    #print("\nВсего времени затрачено: ", round(sum(staffer.values())/3600, 2), 'h\n')
    H = sum(staffer.values())
    hhmm = "{0}:{1:0>2}".format( H//3600, round((H%3600)/60 ))
    print( "{0:>26}  {1:^10} {2:>6}".format("    Итого:", round(H/3600, 2), hhmm ) )
    #print("\nВсего времени затрачено: {0}:{1:0>2}\n".format( sum(staffer.values())//3600, round((sum(staffer.values())%3600)/60) ) ) 

Game
	 WSR-49405 [GS] Логирование работы в группе ЦПП ВК. Март 2017
	 WSR-49876 [GS] Логирование работы в группе ЦПП ВК. АПРЕЛЬ 2017
	 WSR-50144 Апрель 2017, тим 6 - [GS] Логирование работы в группе ЦПП ВК. Месяц 2017
	 WSR-50145 [GS] Логирование работы в группе ЦПП ВК. Апрель 2017
	 WSR-50202 [GS] Логирование работы в группе ЦПП ВК. апрель 2017
                            Время(ч)    ЧЧ:ММ
        Denis Y                9.38      9:23
           Arseniy T           4.22      4:13
     Aleksandr A               2.83      2:50
           Andrey S            2.57      2:34
             Artem M           3.5       3:30
             Vadim I           2.02      2:01
              Artem M          1.43      1:26
               Artur V         2.9       2:54
             Andrei B          9.75      9:45
             Andrei B          1.83      1:50
        Pavel R                5.73      5:44
        Kseniya B              3.1       3:06
            Igor V             8.33      8:20
       R